# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise, Jack Demeter
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.feature_selection import SelectKBest, chi2

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

In [2]:
## Data Pre-Processing
#### We load in individual csv files collected from Rosbag on TurtleBot3 with ROS.

In [3]:
dataFolder = r"C:\Users\jaker\Documents\Experiment3Data-2019-11-21"

dataFiles = [
    r"gMitTile_s15_t8.csv",
    r"gMitTile_s15_t9.csv",
    r"gMitTile_s15_t10.csv",
    r"gTurf_s15_t3.csv",
    r"gTurf_s15_t4.csv",
    r"gTurf_s15_t5.csv",
    r"gTurf_s15_t6.csv",
    r"gTurf_s15_t7.csv",
    r"gTurf_s15_t8.csv",
    r"gTurf_s15_t9.csv",
    r"gTurf_s15_t10.csv",
    r"gArcTile_s15_t3.csv",
    r"gArcTile_s15_t4.csv",
    r"gArcTile_s15_t5.csv",
    r"gArcTile_s15_t6.csv",
    r"gArcTile_s15_t7.csv",
    r"gArcTile_s15_t8.csv",
    r"gArcTile_s15_t9.csv",
    r"gArcTile_s15_t10.csv",
    r"gCarp_s15_t3.csv",
    r"gCarp_s15_t4.csv",
    r"gCarp_s15_t5.csv",
    r"gCarp_s15_t6.csv",
    r"gCarp_s15_t7.csv",
    r"gCarp_s15_t8.csv",
    r"gCarp_s15_t9.csv",
    r"gCarp_s15_t10.csv",
    r"gMitTile_s15_t3.csv",
    r"gMitTile_s15_t4.csv",
    r"gMitTile_s15_t5.csv",
    r"gMitTile_s15_t6.csv",
    r"gMitTile_s15_t7.csv"
]

savePath = "Data-32Series-Delta30-Squared.csv"

In [6]:
""" 
For each data file we:
adjust the index, 
interpolate NaN values,
drop remaining NaN values, 
drop some empty columns.

We then augment the feature space with delta columns, 
and with polynomial columns,
and label the columns with terrain, speed, and trial number
"""
for i in range(len(dataFiles)):
    terrain = dataFiles[i].split('_')[0][1:]
    speed = dataFiles[i].split('_s')[1][:2]
    trial = dataFiles[i].split('_t')[1][0]

    df = pd.read_csv(os.path.join(dataFolder, dataFiles[i]))
    df = df.rename(columns={'Unnamed: 0': 'Seq'})
    df = df.set_index('Seq')

    #print(df.isnull().sum().sum())
    df = df.interpolate(method='polynomial', order=1)
    #print(df.isnull().sum().sum())

    df = df.dropna()
    #print(df.isnull().sum().sum())
    df = df.reset_index().drop(columns=['Seq'])

    df = df.drop(columns=['OdomPosZ', 'OdomOrientX', 'OdomOrientY', 'OdomLinY', 'OdomLinZ', 'OdomAngX', 'OdomAngY'])
    df['OdomPosXY'] = np.sqrt(df.OdomPosX**2 + df.OdomPosY**2)
    df = df.drop(columns=['OdomPosY', 'OdomPosX'])

    df = df.reset_index().drop(columns=['index'])
    df = df.drop(columns=['Sensor', 'Time'])
    df['Speed']=int(speed)
    df['Terrain']=terrain
    df['Trial']=int(trial)

    if i==0:
        mainDf = df.copy(deep=True)
    else:
        mainDf = pd.concat([mainDf, df], axis=0, sort=False)
    print("Added {} of size {} to mainDf. ".format(dataFiles[i], df.shape))
    print("Data series completed: {}/{}".format(i+1,len(dataFiles)))
    print("MainDf is now size {}".format(mainDf.shape))
    print('\n')


Added gMitTile_s15_t8.csv of size (3831, 18) to mainDf. 
Data series completed: 1/32
MainDf is now size (3831, 18)


Added gMitTile_s15_t9.csv of size (4331, 18) to mainDf. 
Data series completed: 2/32
MainDf is now size (8162, 18)


Added gMitTile_s15_t10.csv of size (4853, 18) to mainDf. 
Data series completed: 3/32
MainDf is now size (13015, 18)


Added gTurf_s15_t3.csv of size (6101, 18) to mainDf. 
Data series completed: 4/32
MainDf is now size (19116, 18)


Added gTurf_s15_t4.csv of size (5840, 18) to mainDf. 
Data series completed: 5/32
MainDf is now size (24956, 18)


Added gTurf_s15_t5.csv of size (3352, 18) to mainDf. 
Data series completed: 6/32
MainDf is now size (28308, 18)


Added gTurf_s15_t6.csv of size (5253, 18) to mainDf. 
Data series completed: 7/32
MainDf is now size (33561, 18)


Added gTurf_s15_t7.csv of size (5251, 18) to mainDf. 
Data series completed: 8/32
MainDf is now size (38812, 18)


Added gTurf_s15_t8.csv of size (6668, 18) to mainDf. 
Data series comple

In [7]:
# avoid reloading dataset when working
df = mainDf.copy(deep=True)

In [8]:
df.head(10)

,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXY,Speed,Terrain,Trial
0,0.093628,0.995607,0.149563,0.001384,-0.012936,0.003677,-0.102512,-0.994637,0.014367,-0.002661,0.006917,0.521337,0.494403,10.553940,18.812442,15,MitTile,8
1,0.093710,0.995599,0.149276,0.005066,-0.013177,0.003931,-0.102530,-0.994631,0.019156,0.017027,0.008514,0.788291,0.634463,9.111135,18.812380,15,MitTile,8
2,0.093793,0.995592,0.148988,0.008748,-0.013698,0.004072,-0.102570,-0.994619,0.020220,0.026605,0.014899,0.219069,0.453701,10.493187,18.812318,15,MitTile,8
3,0.093875,0.995584,0.148700,0.012429,-0.012621,0.002504,-0.102654,-0.994628,0.015431,0.030330,0.012770,-0.061052,0.196325,10.284293,18.812256,15,MitTile,8
4,0.093843,0.995587,0.148514,0.008750,-0.011543,0.000936,-0.102737,-0.994637,0.010642,0.034054,0.010642,-0.341174,-0.061052,10.075399,18.812234,15,MitTile,8
5,0.093811,0.995590,0.148328,0.005070,-0.011305,0.000417,-0.102711,-0.994643,-0.009578,0.015963,-0.008514,-0.915184,-0.056264,10.979809,18.812212,15,MitTile,8
6,0.093779,0.995593,0.148142,0.001390,-0.011305,0.000417,-0.102711,-0.994643,-0.009578,0.015963,-0.008514,-0.915184,-0.056264,10.979809,18.812190,15,MitTile,8
7,0.093746,0.995596,0.147956,-0.002290,-0.011117,-0.000264,-0.102601,-0.994656,-0.007449,-0.004257,-0.028733,-1.149217,0.062848,9.466673,18.812168,15,MitTile,8
8,0.093714,0.995599,0.147770,-0.005970,-0.011083,-0.000845,-0.102489,-0.994668,0.009578,-0.037247,-0.028733,-0.572214,0.187945,8.793902,18.812146,15,MitTile,8
9,0.093682,0.995602,0.147584,-0.009650,-0.010887,-0.000046,-0.102468,-0.994672,0.026605,-0.045761,-0.007449,0.708085,-0.201113,10.211270,18.812124,15,MitTile,8


In [75]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(145353, 1506)

### Feature engineering: elimination

In [76]:
# eliminate columns derived from time, and from angular velocity
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Speed' not in x and 'Terrain' not in x and 'Trial' not in x) ]
df = df.drop(columns=columnsToDrop)

In [77]:
df['Terrain'] = df['Terrain'].replace({'Carp':'Turf', 'ArcTile':'MitTile'})

In [78]:
df.head(8)

,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta1Exp3,OdomOrientZDelta11,OdomOrientZDelta11Exp2,OdomOrientZDelta11Exp3,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta21Exp3,OdomOrientZDelta31,OdomOrientZDelta31Exp2,OdomOrientZDelta31Exp3,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta41Exp3,OdomOrientZDelta51,OdomOrientZDelta51Exp2,OdomOrientZDelta51Exp3,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta61Exp3,OdomOrientZDelta71,OdomOrientZDelta71Exp2,OdomOrientZDelta71Exp3,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta81Exp3,OdomOrientZDelta91,OdomOrientZDelta91Exp2,OdomOrientZDelta91Exp3,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta101Exp3,OdomOrientZDelta111,OdomOrientZDelta111Exp2,OdomOrientZDelta111Exp3,OdomOrientZDelta121,OdomOrientZDelta121Exp2,OdomOrientZDelta121Exp3,OdomOrientZDelta131,OdomOrientZDelta131Exp2,OdomOrientZDelta131Exp3,OdomOrientZDelta141,OdomOrientZDelta141Exp2,OdomOrientZDelta141Exp3,OdomOrientZDelta151,OdomOrientZDelta151Exp2,OdomOrientZDelta151Exp3,OdomOrientZDelta161,OdomOrientZDelta161Exp2,OdomOrientZDelta161Exp3,OdomOrientZDelta171,OdomOrientZDelta171Exp2,OdomOrientZDelta171Exp3,OdomOrientZDelta181,OdomOrientZDelta181Exp2,OdomOrientZDelta181Exp3,OdomOrientZDelta191,OdomOrientZDelta191Exp2,OdomOrientZDelta191Exp3,...,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta121Exp3,OdomPosXYDelta131,OdomPosXYDelta131Exp2,OdomPosXYDelta131Exp3,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta141Exp3,OdomPosXYDelta151,OdomPosXYDelta151Exp2,OdomPosXYDelta151Exp3,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta161Exp3,OdomPosXYDelta171,OdomPosXYDelta171Exp2,OdomPosXYDelta171Exp3,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta181Exp3,OdomPosXYDelta191,OdomPosXYDelta191Exp2,OdomPosXYDelta191Exp3,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta201Exp3,OdomPosXYDelta211,OdomPosXYDelta211Exp2,OdomPosXYDelta211Exp3,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta221Exp3,OdomPosXYDelta231,OdomPosXYDelta231Exp2,OdomPosXYDelta231Exp3,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta241Exp3,OdomPosXYDelta251,OdomPosXYDelta251Exp2,OdomPosXYDelta251Exp3,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta261Exp3,OdomPosXYDelta271,OdomPosXYDelta271Exp2,OdomPosXYDelta271Exp3,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta281Exp3,OdomPosXYDelta291,OdomPosXYDelta291Exp2,OdomPosXYDelta291Exp3,OdomPosXYDelta301,OdomPosXYDelta301Exp2,OdomPosXYDelta301Exp3,Speed,Terrain,Trial
0,-0.000021,4.593532e-10,-9.845099e-15,-0.000117,1.369213e-08,-1.602162e-12,-0.000153,2.334641e-08,-3.567221e-12,-0.000310,9.633534e-08,-2.990050e-11,-0.000206,4.228962e-08,-8.696624e-12,-0.000226,5.090268e-08,-1.148447e-11,0.000041,1.700049e-09,7.009580e-14,0.000249,6.188848e-08,1.539624e-11,0.000260,6.770568e-08,1.761723e-11,0.000309,9.531214e-08,2.942540e-11,0.000506,2.555428e-07,1.291801e-10,0.000735,5.402217e-07,3.970618e-10,0.000994,9.888575e-07,9.833329e-10,0.001268,0.000002,2.037097e-09,0.001392,0.000002,2.697768e-09,0.001665,0.000003,4.614042e-09,0.001813,0.000003,5.964037e-09,0.001870,0.000003,6.536602e-09,0.002095,0.000004,9.197385e-09,0.002160,0.000005,1.008332e-08,...,-0.001878,0.000004,-6.622307e-09,-0.002115,0.000004,-9.459604e-09,-0.002290,0.000005,-1.201526e-08,-0.002530,0.000006,-1.618945e-08,-0.002780,0.000008,-2.147848e-08,-0.002953,0.000009,-2.574691e-08,-0.003253,0.000011,-3.442248e-08,-0.003507,0.000012,-4.315035e-08,-0.003737,0.000014,-5.218668e-08,-0.003969,0.000016,-6.250854e-08,-0.004253,0.000018,-7.691240e-08,-0.004460,0.000020,-8.870081e-08,-0.004768,0.000023,-1.083874e-07,-0.005013,0.000025,-1.260139e-07,-0.005302,0.000028,-1.490526e-07,-0.005564,0.000031,-1.722840e-07,-0.005779,0.000033,-1.930108e-07,-0.006097,0.000037,-2.265978e-07,-0.006436,0.000041,-2.666253e-07,15,MitTile,8
1,-0.000021,4.593532e-10,-9.845099e-15,-0.000132,1.753519e-08,-2.322018e-12,-0.000181,3.267875e-08,-5.907423e-12,-0.000301,9.071220e-08,-2.7

### Speed filtering

In [79]:
df = df[df['Speed']==15]

### Logistic Regression
#### Now that we have processed the data, we can apply logistic regression to classify.

In [80]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']<9]
dfTest = df[df['Trial']>=9]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

### Feature engineering: Kbest features

In [81]:
featureCount = 30
test = SelectKBest(k=featureCount)
fit = test.fit(X_train, Y_train)
# print(fit.scores_)
print(X_test.columns.tolist()[fit.scores_.argmax()])
X_train = fit.transform(X_train)
X_test = fit.transform(X_test)

ImuAngVelXDelta21Exp2


In [82]:
# randomly split to train and test
# # separate labels
# terrains = df['Terrain']
# speeds = df['Speed']
# df= df.drop(columns=['Terrain', 'Speed'])

# X = df
# Y = terrains
# X_test, X_train, Y_test, Y_train = train_test_split(X, Y, test_size=0.5)

In [83]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(123978, 30)
(123978,)
(21375, 30)
(21375,)


In [84]:
model = LogisticRegression()
model.fit(X_train, Y_train)

C:\Users\jaker\Anaconda3\envs\tf-p37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [85]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))

Accuracy on test set is: 0.7336608187134503
